<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/python-code-for-FinancialAnalysis-book/blob/master/scrapywiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=180514f1ed4a01b898c326d3638dd2b2aa29b73c1b2cb3056cefb2b1a572808f
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import yfinance as yf
import datetime as dt
import time
import numpy as np
import pandas as pd
from tabulate import tabulate
from google.colab import files

In [ ]:
def get_html(url):
  try:
    response = requests.get(url)
    response.raise_for_status()
    response.encoding = response.apparent_encoding
  except Exception as e:
    print(f'Exception happen: {e}')
  else:
    return response.text

In [ ]:
def parser(html):
  soup  = bs(html,'html.parser')
  tbody = soup.find('tbody')
  resultlist = []
  for tr in tbody.find_all('tr'):
    tdas = tr.find_all('a')
    resultlist.append([tdas[0].string, tdas[1].string])
  
  return resultlist

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
html = get_html(url)
result = parser(html)

In [ ]:
previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
years_ago = dt.date.today()-dt.timedelta(days=5 * 365)
Default_start = years_ago.strftime("%Y-%m-%d")

# request user input
print("Calculate the 4 statistical moments of the rate of returns of the S&P 500 stocks.")
Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
T_interval = input('Enter date interval(1d,1wk,1mo Default = 1mo)-->') or '1mo'

Calculate the 4 statistical moments of the rate of returns of the S&P 500 stocks.
Enter start date(Default = 5 Years ago, 2015-07-23)-->
Enter date interval(1d,1wk,1mo Default = 1mo)-->


In [ ]:
"""get all data"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Data = []
Url = []

for i in range(1,len(result)):

  try:
    Data.append(yf.download(result[i][0].replace(".","-"), start = Start, interval = T_interval)['Adj Close'].dropna())
    Url.append('https://finance.yahoo.com/quote/'+result[i][0])
  except Exception:
    print("Problem retrieving info for ticker: " + result[i][0])  
    continue

  if i%20 == 0:
    r = np.random.randint(3,8)
    time.sleep(r)
    print(str(i) + ' of 500   Getting S&P 500 data ...')

Data = pd.concat(Data,axis=1)

In [ ]:
Ret = Data.pct_change()
summury = pd.concat([Ret.mean(),Ret.std(),Ret.skew(),Ret.kurtosis(),np.divide(Ret.std(),Ret.mean())],axis=1)
summury.columns = ['Mean','standard deviation','skewness','kurtosis','coefficient of variation']
Ticker  = [i[0] for i in result]
Ticker = Ticker[1:]
description = [i[1] for i in result]
description = description[1:]
summury.insert(0, "description", description, True) 
summury.insert(0, "Stock Name", Ticker, True) 
summury.insert(6, "Url", Url, True) 

In [ ]:
#There are more then 500 tickers in the S&P500 Index
print("")
print("                                                                                First Four Moments of Rates of Returns")
print("                                                                                       S&P 500 ")
print ("                                                                                       " + Start + ' to ' + Default_end            )
print("")
print(tabulate(summury,headers=summury.columns,tablefmt="github",showindex=False))
print("")
print("S&P 500 List Source: https://www.slickcharts.com/sp500")


                                                                                First Four Moments of Rates of Returns
                                                                                       S&P 500 
                                                                                       2015-07-23 to 2020-07-20

| Stock Name   | description                            |          Mean |   standard deviation |    skewness |    kurtosis | Url                                   |   coefficient of variation |
|--------------|----------------------------------------|---------------|----------------------|-------------|-------------|---------------------------------------|----------------------------|
| MMM          | 3M Company                             |   0.00586622  |            0.0575933 | -0.555271   |  0.222614   | https://finance.yahoo.com/quote/MMM   |                   9.81779  |
| ABT          | Abbott Laboratories                    |   0.0163782   |            0.06

In [ ]:
summury.to_csv('chapter19_SP500FourMoments.csv')
print("Saving SP500 report to 'Download' folder") 
files.download('chapter19_SP500FourMoments.csv')
print("Finish saving SP500 report to 'Download' folder")

Saving SP500 report to 'Download' folder


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finish saving SP500 report to 'Download' folder
